## Data processing

Steps
1. Load cvs files
2. Merge files
3. correct data types
4. inputate NA
5. Feature engeneering


Problems
date matching missing data for: only working for 2017
    kiwo dates 
    holiday 

Import libraries

In [37]:
import pandas as pd
import datetime

1. Load cvs files

In [54]:
# Load a CSV files
df_kiwo = pd.read_csv('/workspaces/bakery_prediction/0_DataPreparation/Raw/kiwo.csv')
df_daten = pd.read_csv('/workspaces/bakery_prediction/0_DataPreparation/Raw/umsatzdaten_gekuerzt.csv')
df_wetter = pd.read_csv('/workspaces/bakery_prediction/0_DataPreparation/Raw/wetter.csv')

In [55]:
#converts to date and sets index
df_kiwo['Datum'] = pd.to_datetime(df_kiwo['Datum'], format='%Y-%m-%d')

df_daten['Datum'] = pd.to_datetime(df_daten['Datum'], format='%Y-%m-%d')

df_wetter['Datum'] = pd.to_datetime(df_wetter['Datum'], format='%Y-%m-%d')


Creates id ang warengruppe for day with Umsatz 0 and for test dates

In [ ]:
warengruppe6_dates = pd.to_datetime([
    '10-24' '10-25' '10-26' '10-27' '10-29' '10-30' '10-31' '11-01' '11-02' '11-03' '11-04' '11-05' '11-06' '11-07' '11-08' '11-09' '11-10' '11-11' '11-12' '11-13' '11-14' '11-15' '11-16' '11-17' '11-18' '11-19' '11-20' '11-21' '11-22' '11-23' '11-24' '11-25' '11-26' '11-27' '11-28' '11-29' '11-30' '12-01' '12-02' '12-03' '12-04' '12-05' '12-06' '12-07' '12-08' '12-09' '12-10' '12-11' '12-12' '12-13' '12-14' '12-15' '12-16' '12-17''12-18' '12-19' '12-20' '12-21' '12-22' '12-23' '12-24' '12-27' '12-29' '12-30' '01-02' '10-28' '12-28' '01-03' '01-04' '01-05' '01-06' '12-31'])

dates_umsatz_null = pd.to_datetime([
    "2013-12-25", "2013-12-26", "2014-01-01",
    "2014-04-18", "2014-05-01", "2014-05-03",
    "2014-05-04", "2014-08-17", "2014-12-25",
    "2014-12-26", "2015-01-01", "2015-04-03",
    "2015-05-01", "2015-12-25", "2015-12-26",
    "2016-01-01", "2016-03-24", "2016-03-25",
    "2016-07-07", "2016-07-18", "2016-08-14",
    "2016-08-15", "2016-12-17", "2016-12-25",
    "2016-12-26", "2017-01-01", "2017-04-14",
    "2017-05-01", "2017-07-04", "2017-10-23",
    "2017-10-31", "2017-12-25", "2017-12-26",
    "2018-01-01", "2018-03-30", "2018-05-01",
    "2018-05-21"
])

def prepare_dataset(df_daten, warengruppe6_dates=None, dates_umsatz_null=None):
    """
    Prepara o dataset para modelagem:
    - Expande todas as datas em Warengruppe 1..5 ou 1..6 (se estiver em warengruppe6_dates)
    - Replica features diárias para cada Warengruppe
    - Junta Umsatz original (quando existir)
    - Coloca Umsatz = 0 para datas em dates_umsatz_null ou sem venda
    - Cria coluna id = YYMMDD + Warengruppe
    """
    
    df = df_inputated.copy()
    
    # Garantir tipo datetime
    df["Datum"] = pd.to_datetime(df["Datum"])
    
    # Normalizar listas de datas (podem vir como lista de strings)
    if warengruppe6_dates is None:
        warengruppe6_dates = pd.to_datetime([])
    else:
        warengruppe6_dates = pd.to_datetime(warengruppe6_dates)
    
    if dates_umsatz_null is None:
        dates_umsatz_null = pd.to_datetime([])
    else:
        dates_umsatz_null = pd.to_datetime(dates_umsatz_null)
    
    # --- 1) Features por data (clima, KielerWoche etc.) ---
    # Vamos pegar todas as colunas exceto Warengruppe, Umsatz e id
    cols_date_level = [c for c in df.columns if c not in ["Warengruppe", "Umsatz", "id"]]
    
    # Um registro por data (assumindo que clima, feriado, etc. é igual no dia todo)
    date_features = df[cols_date_level].drop_duplicates(subset=["Datum"])
    
    # Lista de datas únicas
    unique_dates = date_features["Datum"].drop_duplicates().sort_values()
    
    # --- 2) Criar grade completa Datum x Warengruppe ---
    rows = []
    for date in unique_dates:
        if date in warengruppe6_dates:
            groups = range(1, 7)   # 1..6
        else:
            groups = range(1, 6)   # 1..5
        
        for g in groups:
            rows.append({"Datum": date, "Warengruppe": g})
    
    full_grid = pd.DataFrame(rows)
    
    # --- 3) Juntar features diárias na grade ---
    df_expanded = full_grid.merge(date_features, on="Datum", how="left")
    
    # --- 4) Juntar Umsatz original (por Datum + Warengruppe) ---
    if "Warengruppe" in df.columns:
        umsatz_source = df[["Datum", "Warengruppe", "Umsatz"]].copy()
        df_expanded = df_expanded.merge(
            umsatz_source,
            on=["Datum", "Warengruppe"],
            how="left",
            suffixes=("", "_orig")
        )
    else:
        # se não existir Warengruppe no original, só cria a coluna
        df_expanded["Umsatz"] = pd.NA
    
    # --- 5) Tratar Umsatz = 0 para datas sem venda ---
    # Para datas explicitamente sem venda
    df_expanded.loc[df_expanded["Datum"].isin(dates_umsatz_null), "Umsatz"] = 0
    
    # Qualquer NaN restante também vira 0
    df_expanded["Umsatz"] = df_expanded["Umsatz"].fillna(0)
    
    # --- 6) Criar coluna id ---
    df_expanded["id"] = (
        df_expanded["Datum"].dt.strftime("%y%m%d") +
        df_expanded["Warengruppe"].astype(int).astype(str)
    )
    
    # --- 7) Ordenar e retornar ---
    df_expanded = df_expanded.sort_values(["Datum", "Warengruppe"]).reset_index(drop=True)
    
    return df_expanded


2. Merge files

In [58]:
# merge dataframes on a common column
df_kiwowetter = pd.merge(df_daten, df_kiwo, on='Datum', how='outer')

df_merged = pd.merge(df_kiwowetter, df_wetter, on='Datum', how='outer')

In [59]:
# Display the first 5 rows
print(df_merged.head())

   id      Datum  Warengruppe  Umsatz  KielerWoche  Bewoelkung  Temperatur  \
0 NaN 2012-01-01          NaN     NaN          NaN         8.0      9.8250   
1 NaN 2012-01-02          NaN     NaN          NaN         7.0      7.4375   
2 NaN 2012-01-03          NaN     NaN          NaN         8.0      5.5375   
3 NaN 2012-01-04          NaN     NaN          NaN         4.0      5.6875   
4 NaN 2012-01-05          NaN     NaN          NaN         6.0      5.3000   

   Windgeschwindigkeit  Wettercode  
0                 14.0        58.0  
1                 12.0         NaN  
2                 18.0        63.0  
3                 19.0        80.0  
4                 23.0        80.0  


In [60]:
# split dataset based on datum from 01.07.2013 to 31.07.2019.
df_merged = df_merged[(df_merged['Datum'] >= '2013-07-01') & (df_merged['Datum'] <= '2019-07-31')]



3. correct data types

In [61]:
#descriptive statistics
print(df_merged.describe())

#data types and non-null counts
print(df_merged.info())

#missing values in each column
print(df_merged.isnull().sum())

                 id                          Datum  Warengruppe       Umsatz  \
count  9.334000e+03                           9724  9334.000000  9334.000000   
mean   1.559311e+06  2016-02-23 14:29:16.314273792     3.088172   206.749044   
min    1.307011e+06            2013-07-01 00:00:00     1.000000     7.051201   
25%    1.410123e+06            2014-10-29 00:00:00     2.000000    96.897441   
50%    1.601102e+06            2016-02-14 12:00:00     3.000000   161.900831   
75%    1.704223e+06            2017-06-15 00:00:00     4.000000   280.644663   
max    1.807315e+06            2019-07-31 00:00:00     6.000000  1879.461831   
std    1.512503e+05                            NaN     1.489002   144.545189   

       KielerWoche  Bewoelkung   Temperatur  Windgeschwindigkeit   Wettercode  
count        232.0  9653.00000  9708.000000          9708.000000  7337.000000  
mean           1.0     4.72796    12.021734            10.992274    36.716369  
min            1.0     0.00000    -8.47

In [62]:
# set warengruppe, bewoelkung, windgeschwindigkeit, wetter code and kieler woche as integer
df_merged['Warengruppe'] = df_merged['Warengruppe'].astype('Int64')
df_merged['Bewoelkung'] = df_merged['Bewoelkung'].astype('Int64')
df_merged['Windgeschwindigkeit'] = df_merged['Windgeschwindigkeit'].astype('Int64')
df_merged['Wettercode'] = df_merged['Wettercode'].astype('Int64')
df_merged['KielerWoche'] = df_merged['KielerWoche'].astype('Int64')

Save to csv// import from csv

In [63]:
#save merged dataframe to a new CSV file
df_merged.to_csv('/workspaces/bakery_prediction/0_DataPreparation/Processed/merged_data.csv', index=False)

In [77]:
df_merged = pd.read_csv('/workspaces/bakery_prediction/0_DataPreparation/Processed/merged_data.csv')

4. inputate NA

In [112]:
df_inputated = df_merged.copy()

In [130]:
#missing values in each column
print(df_inputated.isnull().sum())

id                       38
Datum                     0
Warengruppe              38
Umsatz                 1798
KielerWoche            1760
Bewoelkung             1760
Temperatur             1760
Windgeschwindigkeit    1760
Wettercode             4092
dtype: int64


In [133]:
# Quantos NaN em Warengruppe dentro da máscara?
mask = (df_merged['Datum'] >= '2018-08-01') & (df_merged['Datum'] <= '2019-07-31')

print("NaN dentro da máscara:", df_merged.loc[mask, "Warengruppe"].isna().sum())
print("NaN fora da máscara:", df_merged.loc[~mask, "Warengruppe"].isna().sum())

NaN dentro da máscara: 352
NaN fora da máscara: 38


Categorical Wettercode per week mode

In [ ]:
"""# derive week and month for df_inputated
df_inputated['Woche'] = pd.to_datetime(df_inputated['Datum']).dt.isocalendar().week
df_inputated['Monat'] = pd.to_datetime(df_inputated['Datum']).dt.month

# precompute weekly and monthly modes (ignore NaNs when computing mode)
weekly_mode_map = df_inputated.groupby('Woche')['Wettercode'].agg(
    lambda s: s.dropna().mode().iloc[0] if not s.dropna().mode().empty else pd.NA
).to_dict()

monthly_mode_map = df_inputated.groupby('Monat')['Wettercode'].agg(
    lambda s: s.dropna().mode().iloc[0] if not s.dropna().mode().empty else pd.NA
).to_dict()

# fill missing Wettercode: use weekly mode, if not available fallback to monthly mode
def fill_wettercode_row(row):
    if pd.isna(row['Wettercode']):
        w_mode = weekly_mode_map.get(row['Woche'], pd.NA)
        if not pd.isna(w_mode):
            return w_mode
        return monthly_mode_map.get(row['Monat'], pd.NA)
    return row['Wettercode']

df_inputated['Wettercode'] = df_inputated.apply(fill_wettercode_row, axis=1)"""

In [ ]:
'''# 1) Compare antes vs depois
print("ANTES:", df_merged['KielerWoche'].value_counts(dropna=False))
print("DEPOIS:", df_inputated['KielerWoche'].value_counts(dropna=False))

# 2) Quais linhas viraram 0?
flip = (df_merged['KielerWoche'] == 1) & (df_inputated['KielerWoche'] == 0)
print("Flips:", flip.sum())
print(df_merged.loc[flip, ['Datum']].assign(KW_before=1).head())

# 3) Veja se há múltiplas colunas de KielerWoche após os merges
[k for k in df_merged.columns if 'KielerWoche' in k]
[k for k in df_inputated.columns if 'KielerWoche' in k]'''


set not kiwo to 0 

In [114]:
#fix kieler woche missing values by filling with 0
df_inputated['KielerWoche'].fillna(0, inplace=True)

#set 'KielerWoche' as integer (nullable int64 to allow current NaNs)
df_inputated['KielerWoche'] = df_inputated['KielerWoche'].astype('int64')

/tmp/ipykernel_3163/3859340180.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_inputated['KielerWoche'].fillna(0, inplace=True)


missing Bewoelkung, Temperatur and/or Windgeschwindigkeit
Inputated with mean of day after and before

In [115]:
#what days are missing Bewoelkung, Temperatur and/or     Windgeschwindigkeit
missing_weather = df_inputated[df_inputated[['Bewoelkung', 'Temperatur', 'Windgeschwindigkeit']].isnull().any(axis=1)]
print(missing_weather[[ 'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit']])

      Bewoelkung  Temperatur  Windgeschwindigkeit
6371         NaN         NaN                  NaN
6372         NaN         NaN                  NaN
6373         NaN         NaN                  NaN
6374         NaN         NaN                  NaN
6375         NaN         NaN                  NaN
...          ...         ...                  ...
8019         NaN       8.625                  9.0
8020         NaN       8.625                  9.0
8021         NaN       8.625                  9.0
8022         NaN       8.625                  9.0
8023         NaN       8.625                  9.0

[71 rows x 3 columns]


fill Bewoelkung, Temperatur, Windgeschwindigkeit to mean of day before and after

In [116]:
#fill missing weather data with the mean of day before and after
df_inputated['Bewoelkung'] = df_inputated['Bewoelkung'].interpolate()
df_inputated['Temperatur'] = df_inputated['Temperatur'].interpolate()
df_inputated['Windgeschwindigkeit'] = df_inputated['Windgeschwindigkeit'].interpolate()



In [117]:
# set warengruppe, bewoelkung, windgeschwindigkeit, wetter code and kieler woche as integer
df_inputated['Bewoelkung'] = df_inputated['Bewoelkung'].astype('int64')
df_inputated['Windgeschwindigkeit'] = df_inputated['Windgeschwindigkeit'].astype('int64')

In [118]:
# median values before and after imputation from 2016-12-11 to 2017-11-08
df_merged_ = df_merged[(df_merged['Datum'] >= '2016-12-11') & (df_merged['Datum'] <= '2017-11-08')]
df_inputated_ = df_inputated[(df_inputated['Datum'] >= '2016-12-11') & (df_inputated['Datum'] <= '2017-11-08')]

print("Mean Bewoelkung before imputation:", df_merged_['Bewoelkung'].mean())
print("Mean Bewoelkung after imputation:", df_inputated_['Bewoelkung'].mean())
print("Mean Temperatur before imputation:", df_merged_['Temperatur'].mean())
print("Mean Temperatur after imputation:", df_inputated['Temperatur'].mean())
print("Mean Windgeschwindigkeit before imputation:", df_merged['Windgeschwindigkeit'].mean())
print("Mean Windgeschwindigkeit after imputation:", df_inputated['Windgeschwindigkeit'].mean())

Mean Bewoelkung before imputation: 4.922250316055626
Mean Bewoelkung after imputation: 4.868723532970357
Mean Temperatur before imputation: 12.174979272449603
Mean Temperatur after imputation: 12.020863154668861
Mean Windgeschwindigkeit before imputation: 10.992274412855377
Mean Windgeschwindigkeit after imputation: 10.99156725627314


Discrepancies of Bewoelkung, Temperatur and/or     Windgeschwindigkeit for the same day across WarenGruppe

In [119]:
# check if there are discrepancies for the same date for the columns 'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit'
#for 'Warengruppe' 6 the temperature is different for the same date as the other 'Warengruppe'
discrepancies = df_inputated.groupby('Datum').agg({
    'Bewoelkung': pd.Series.nunique,
    'Temperatur': pd.Series.nunique,
    'Windgeschwindigkeit': pd.Series.nunique
}).reset_index() 
discrepancies = discrepancies[(discrepancies['Bewoelkung'] > 1) | (discrepancies['Temperatur'] > 1) | (discrepancies['Windgeschwindigkeit'] > 1)]
print("Discrepancies in weather data for the same date:")
print(discrepancies)

Discrepancies in weather data for the same date:
           Datum  Bewoelkung  Temperatur  Windgeschwindigkeit
1259  2016-12-11           1           6                    6
1556  2017-10-04           2           5                    1
1557  2017-10-05           2           5                    1
1587  2017-11-04           2           1                    1
1589  2017-11-06           2           1                    1


uses value of WarenGruppe 1 to all groups 

In [128]:
cols = ['Bewoelkung', 'Temperatur', 'Windgeschwindigkeit']

for date in discrepancies['Datum']:
    weather_values = df_inputated.loc[df_inputated['Datum'] == date, ['Warengruppe'] + cols]
    reference_values = weather_values.loc[weather_values['Warengruppe'] == 1, cols].iloc[0]
    # >>> use .values para evitar alinhamento por rótulo <<<
    df_inputated.loc[df_inputated['Datum'] == date, cols] = reference_values.values


For from 01.08.2018 to 31.07.2019 set id and Warengroup

In [121]:
#check for what month and days (no year) warengruppe 6 is avalable every year
mask = df_inputated['Warengruppe'] == 6
# ensure the 'Datum' values are datetime-like before using .dt (do not overwrite original column)
date_series = pd.to_datetime(df_inputated.loc[mask, 'Datum'], errors='coerce')
warengruppe6_dates = date_series.dt.strftime('%m-%d').dropna().unique()
print("Warengruppe 6 is available on the following month-day combinations:")
print(warengruppe6_dates)

Warengruppe 6 is available on the following month-day combinations:
['10-24' '10-25' '10-26' '10-27' '10-29' '10-30' '10-31' '11-01' '11-02'
 '11-03' '11-04' '11-05' '11-06' '11-07' '11-08' '11-09' '11-10' '11-11'
 '11-12' '11-13' '11-14' '11-15' '11-16' '11-17' '11-18' '11-19' '11-20'
 '11-21' '11-22' '11-23' '11-24' '11-25' '11-26' '11-27' '11-28' '11-29'
 '11-30' '12-01' '12-02' '12-03' '12-04' '12-05' '12-06' '12-07' '12-08'
 '12-09' '12-10' '12-11' '12-12' '12-13' '12-14' '12-15' '12-16' '12-17'
 '12-18' '12-19' '12-20' '12-21' '12-22' '12-23' '12-24' '12-27' '12-29'
 '12-30' '01-02' '10-28' '12-28' '01-03' '01-04' '01-05' '01-06' '12-31']


In [122]:
# máscara fornecida
mask = (df_inputated['Datum'] >= '2018-08-01') & (df_inputated['Datum'] <= '2019-07-31')

# lista das datas que devem ter 6 grupos

# garante que Datum é datetime
df_inputated["Datum"] = pd.to_datetime(df_inputated["Datum"])

# separa o bloco que será expandido
df_to_expand = df_inputated.loc[mask, ["Datum"]].drop_duplicates()

def expand_groups(row):
    date = row["Datum"]
    if date in warengruppe6_dates:
        grupos = range(1, 7)   # 1..6
    else:
        grupos = range(1, 6)   # 1..5
        
    return pd.DataFrame({
        "Datum": [date] * len(grupos),
        "Warengruppe": grupos
    })

# gera linhas expandidas
expanded_rows = pd.concat(df_to_expand.apply(expand_groups, axis=1).to_list(), ignore_index=True)

# cria ID (YYMMDD + warengruppe)
expanded_rows["YYMMDD"] = expanded_rows["Datum"].dt.strftime("%y%m%d")
expanded_rows["id"] = expanded_rows["YYMMDD"] + expanded_rows["Warengruppe"].astype(str)
expanded_rows.drop(columns=["YYMMDD"], inplace=True)

# remove as linhas originais que estavam dentro da máscara
df_inputated = df_inputated.loc[~mask]

# adiciona as novas linhas expandidas
df_inputated = pd.concat([df_inputated, expanded_rows], ignore_index=True)

# reordena se quiser
df_inputated = df_inputated.sort_values(by=["Datum", "Warengruppe"], ignore_index=True)

df_inputated.head()


,id,Datum,Warengruppe,Umsatz,KielerWoche,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode
0,1307011.0,2013-07-01,1.0,148.828353,0.0,6.0,17.8375,15.0,20.0
1,1307012.0,2013-07-01,2.0,535.856285,0.0,6.0,17.8375,15.0,20.0
2,1307013.0,2013-07-01,3.0,201.198426,0.0,6.0,17.8375,15.0,20.0
3,1307014.0,2013-07-01,4.0,65.890169,0.0,6.0,17.8375,15.0,20.0
4,1307015.0,2013-07-01,5.0,317.475875,0.0,6.0,17.8375,15.0,20.0


In [ ]:
# exemplo da sua lista
dates_umsatz_null = pd.to_datetime([
    "2013-12-25", "2013-12-26", "2014-01-01",
    "2014-04-18", "2014-05-01", "2014-05-03",
    "2014-05-04", "2014-08-17", "2014-12-25",
    "2014-12-26", "2015-01-01", "2015-04-03",
    "2015-05-01", "2015-12-25", "2015-12-26",
    "2016-01-01", "2016-03-24", "2016-03-25",
    "2016-07-07", "2016-07-18", "2016-08-14",
    "2016-08-15", "2016-12-17", "2016-12-25",
    "2016-12-26", "2017-01-01", "2017-04-14",
    "2017-05-01", "2017-07-04", "2017-10-23",
    "2017-10-31", "2017-12-25", "2017-12-26",
    "2018-01-01", "2018-03-30", "2018-05-01",
    "2018-05-21"
])

# garantir datetime
df_inputated["Datum"] = pd.to_datetime(df_inputated["Datum"])

# manter apenas datas únicas
unique_dates = df_inputated["Datum"].drop_duplicates()

# função de expansão
def expand(row):
    date = row["Datum"]
    if date in warengruppe6_dates:
        groups = range(1, 7)   # 1..6
    else:
        groups = range(1, 6)   # 1..5

    return pd.DataFrame({
        "Datum": [date] * len(groups),
        "Warengruppe": groups
    })

# aplica expansão
expanded = pd.concat(
    unique_dates.to_frame().apply(expand, axis=1).to_list(),
    ignore_index=True
)

# agora juntar VENDAS
# merge com o original para obter Umsatz onde existir
expanded = expanded.merge(
    df_inputated[["Datum", "Umsatz"]],
    on="Datum",
    how="left"
)

# preencher vendas faltantes com 0
expanded["Umsatz"].fillna(0, inplace=True)

# criar id = YYMMDD + Warengruppe
expanded["id"] = (
    expanded["Datum"].dt.strftime("%y%m%d") +
    expanded["Warengruppe"].astype(str)
)

# esse agora é seu df final
df_inputated = expanded.sort_values(["Datum", "Warengruppe"], ignore_index=True)

df_inputated.head()


Save to csv// import from csv

In [123]:
#save inputated dataframe to a new CSV file

df_inputated.to_csv('/workspaces/bakery_prediction/0_DataPreparation/Processed/inputated_data.csv', index=False)

In [ ]:
df_inputated = pd.read_csv('/workspaces/bakery_prediction/0_DataPreparation/Processed/inputated_data.csv')

5. Feature engeneering

In [38]:
df_featured = df_inputated.copy() 

insert day of the week

In [39]:
#insert new column day of week as integer (1=Monday,...,7=Sunday)   
df_featured['Wochentag'] = pd.to_datetime(df_featured['Datum']).dt.dayofweek + 1
print(df_featured[['Datum', 'Wochentag']].head())


        Datum  Wochentag
0  2013-07-01          1
1  2013-07-02          2
2  2013-07-03          3
3  2013-07-04          4
4  2013-07-05          5


insert holiday

In [43]:
#insert new column holiday from 2013-07-01 to 2018-07-31 
holidays = [
    '2013-10-03', '2013-12-25', '2013-12-26',
    '2014-01-01', '2014-04-18', '2014-04-21',
    '2014-05-01', '2014-05-29', '2014-06-09',
    '2014-10-03', '2014-12-25', '2014-12-26',
    '2015-01-01', '2015-04-03', '2015-04-06',
    '2015-05-01', '2015-05-14', '2015-05-25',
    '2015-10-03', '2015-12-25', '2015-12-26',
    '2016-01-01', '2016-03-25', '2016-03-28',
    '2016-05-01', '2016-05-05', '2016-05-16',
    '2016-10-03', '2016-12-25', '2016-12-26',
    '2017-01-01', '2017-04-14', '2017-04-17',
    '2017-05-01', '2017-05-25', '2017-06-05',
    '2017-10-03', '2017-10-31', '2017-12-25', '2017-12-26',
    '2018-01-01', '2018-03-30', '2018-04-02',
    '2018-05-01', '2018-05-10', '2018-05-21'
]
df_featured['Feiertag'] = df_featured['Datum'].isin(holidays)
#convert boolean to integer (0 not holiday, 1 holiday)
df_featured['Feiertag'] = df_featured['Feiertag'].astype(int)

print(df_featured[['Datum', 'Feiertag']].head())

        Datum  Feiertag
0  2013-07-01         0
1  2013-07-02         0
2  2013-07-03         0
3  2013-07-04         0
4  2013-07-05         0


set season from day and month

In [44]:
import datetime

# create column season like this: Winter from 22 December to 20 march, Spring from 21 March to 21 june, Summer from 22 June to 21 september, Autumn from 22 September to 21 december
def get_season(date_or_month):
    # If a date-like is passed (string/Timestamp/date), use exact day-month boundaries
    if isinstance(date_or_month, (str, pd.Timestamp, datetime.date)):
        d = pd.to_datetime(date_or_month)
        m, day = d.month, d.day
        if (m == 12 and day >= 22) or m in (1, 2) or (m == 3 and day <= 20):
            return 1 #'Winter'
        if (m == 3 and day >= 21) or m in (4, 5) or (m == 6 and day <= 21):
            return 2 #'Spring'
        if (m == 6 and day >= 22) or m in (7, 8) or (m == 9 and day <= 21):
            return 3 #'Summer'
        return 4 #'Autumn'
    # If an integer month is passed, fall back to month-based mapping
    m = int(date_or_month)
    if m in (12, 1, 2):
        return 1 #'Winter'
    if m in (3, 4, 5):
        return 2 #'Spring'
    if m in (6, 7, 8):
        return 3 #'Summer'
    return 4 #'Autumn'

df_featured['Monat'] = pd.to_datetime(df_featured['Datum']).dt.month
# Derive season based on the full date (day-month boundaries are respected)
df_featured['Jahreszeit'] = pd.to_datetime(df_featured['Datum']).apply(get_season)
print(df_featured[['Datum', 'Monat', 'Jahreszeit']].head())


        Datum  Monat  Jahreszeit
0  2013-07-01      7           3
1  2013-07-02      7           3
2  2013-07-03      7           3
3  2013-07-04      7           3
4  2013-07-05      7           3


In [48]:
# insert new column for school holidays (ferien)
school_holidays = [
    # Summer holidays
    ('2013-07-22', '2013-08-31'),
    ('2014-07-21', '2014-08-31'),
    ('2015-07-20', '2015-08-31'),
    ('2016-07-18', '2016-08-31'),
    ('2017-07-17', '2017-08-31'),
    ('2018-07-16', '2018-08-31'),
    # Autumn holidays
    ('2013-10-14', '2013-10-26'),
    ('2014-10-13', '2014-10-25'),
    ('2015-10-12', '2015-10-24'),
    ('2016-10-10', '2016-10-22'),
    ('2017-10-09', '2017-10-21'),
    ('2018-10-08', '2018-10-20'),
    # Christmas holidays
    ('2013-12-23', '2014-01-05'),
    ('2014-12-22', '2015-01-04'),
    ('2015-12-21', '2016-01-03'),
    ('2016-12-23', '2017-01-06'),
    ('2017-12-22', '2018-01-05'),
    # Winter holidays
    ('2014-02-03', '2014-02-15'),
    ('2015-02-02', '2015-02-14'),
    ('2016-02-01', '2016-02-13'),
    ('2017-01-30', '2017-02-11'),
    ('2018-01-29', '2018-02-10'),
    # Easter holidays
    ('2013-03-25', '2013-04-06'),
    ('2014-04-14', '2014-04-25'),
    ('2015-04-06', '2015-04-18'),
    ('2016-03-21', '2016-04-01'),
    ('2017-04-10', '2017-04-21'),
    ('2018-03-26', '2018-04-06'),
    # Spring holidays
    ('2013-05-20', '2013-05-31'),
    ('2014-05-19', '2014-05-30'),
    ('2015-05-18', '2015-05-29'),
    ('2016-05-16', '2016-05-27'),
    ('2017-05-15', '2017-05-26'),
    ('2018-05-14', '2018-05-25'),
]       
df_featured['Ferien'] = 0
for start, end in school_holidays:
    mask = (pd.to_datetime(df_featured['Datum']) >= pd.to_datetime(start)) & (pd.to_datetime(df_featured['Datum']) <= pd.to_datetime(end))
    df_featured.loc[mask, 'Ferien'] = 1     



In [49]:
#save featured dataframe to a new CSV file
df_featured.to_csv(r'C:\Users\giuli\Documents\Open_Campus\bakery_prediction\0_DataPreparation\Processed\featured_data.csv', index=False)